<h2> Imports & Configuration </h2>

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [ ]:
import os
import sys
import pandas as pd
os.environ['JAVA_HOME'] = r'C:\Program Files\Java\jdk-17'
#os.environ['PYSPARK_HOME'] = r'C:\Users\lpaschoal\pyspark\spark-4.0.1-bin-hadoop3'
os.environ['SPARK_HOME'] = r'C:\Users\lpaschoal\pyspark\spark-4.0.1-bin-hadoop3'
os.environ['HADOOP_HOME'] = r'C:\Users\lpaschoal\pyspark\hadoop\winutils\hadoop-3.3.6'

sys.path.insert(0, os.getenv('SPARK_HOME') + r"\python\pyspark")
sys.path.insert(0, os.getenv('SPARK_HOME') + r"\python\lib\pyspark.zip")
sys.path.insert(0, os.getenv('SPARK_HOME') + r"\python\lib\py4j-0.10.9.9-src.zip")

from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [4]:
spark.conf.set("spark.sql.shuffle.partitions", "3")
spark.conf.get("spark.sql.shuffle.partitions")
spark.conf.set("spark.sql.adaptive.enabled", "false")
spark.conf.set("spark.python.worker.connectionTimeout", "300s")

'3'

<h2> Simulating Skewed Join </h2>

In [5]:
df_uniform = spark.createDataFrame([i for i in range(1000000)], IntegerType())
df_uniform.show(5, False)

Py4JJavaError: An error occurred while calling o42.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0) (DESKTOP-VJCFDE0 executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:252)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:143)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:158)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:178)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:261)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:70)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:374)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:338)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartit

In [ ]:
(
    df_uniform
    .withColumn("partition", F.spark_partition_id())
    .groupBy("partition")
    .count()
    .orderBy("partition")
    .show(15, False)
)

+---------+-----+
|partition|count|
+---------+-----+
|0        |82944|
|1        |82944|
|2        |83968|
|3        |82944|
|4        |83968|
|5        |82944|
|6        |82944|
|7        |83968|
|8        |82944|
|9        |83968|
|10       |82944|
|11       |83520|
+---------+-----+



In [ ]:
df0 = spark.createDataFrame([0] * 999990, IntegerType()).repartition(1)
df1 = spark.createDataFrame([1] * 15, IntegerType()).repartition(1)
df2 = spark.createDataFrame([2] * 10, IntegerType()).repartition(1)
df3 = spark.createDataFrame([3] * 5, IntegerType()).repartition(1)
df_skew = df0.union(df1).union(df2).union(df3)
df_skew.show(5, False)


[Stage 5:>                (0 + 12) / 12][Stage 6:>                 (0 + 0) / 12]



+-----+
|value|
+-----+
|0    |
|0    |
|0    |
|0    |
|0    |
+-----+
only showing top 5 rows



In [ ]:
(
    df_skew
    .withColumn("partition", F.spark_partition_id())
    .groupBy("partition")
    .count()
    .orderBy("partition")
    .show()
)

+---------+------+
|partition| count|
+---------+------+
|        0|999990|
|        1|    15|
|        2|    10|
|        3|     5|
+---------+------+



In [ ]:
df_joined_c1 = df_skew.join(df_uniform, "value", 'inner')

In [ ]:
(
    df_joined_c1
    .withColumn("partition", F.spark_partition_id())
    .groupBy("partition")
    .count()
    .show(5, False)
)

+---------+-------+
|partition|count  |
+---------+-------+
|0        |1000005|
|1        |15     |
+---------+-------+




[Stage 20:======================================>                   (2 + 1) / 3]



<h2> Simulating Uniform Distribution Through Salting </h2>

In [ ]:
SALT_NUMBER = int(spark.conf.get("spark.sql.shuffle.partitions"))
SALT_NUMBER

3

In [ ]:
df_skew = df_skew.withColumn("salt", (F.rand() * SALT_NUMBER).cast("int"))

In [ ]:
df_skew.show(10, truncate=False)

+-----+----+
|value|salt|
+-----+----+
|0    |2   |
|0    |0   |
|0    |0   |
|0    |1   |
|0    |2   |
|0    |2   |
|0    |0   |
|0    |2   |
|0    |2   |
|0    |1   |
+-----+----+
only showing top 10 rows



In [ ]:
df_uniform = (
    df_uniform
    .withColumn("salt_values", F.array([F.lit(i) for i in range(SALT_NUMBER)]))
    .withColumn("salt", F.explode(F.col("salt_values")))
)

In [ ]:
df_uniform.show(10, truncate=False)

+-----+-----------+----+
|value|salt_values|salt|
+-----+-----------+----+
|0    |[0, 1, 2]  |0   |
|0    |[0, 1, 2]  |1   |
|0    |[0, 1, 2]  |2   |
|1    |[0, 1, 2]  |0   |
|1    |[0, 1, 2]  |1   |
|1    |[0, 1, 2]  |2   |
|2    |[0, 1, 2]  |0   |
|2    |[0, 1, 2]  |1   |
|2    |[0, 1, 2]  |2   |
|3    |[0, 1, 2]  |0   |
+-----+-----------+----+
only showing top 10 rows



In [ ]:
df_joined = df_skew.join(df_uniform, ["value", "salt"], 'inner')

In [ ]:
(
    df_joined
    .withColumn("partition", F.spark_partition_id())
    .groupBy("value", "partition")
    .count()
    .orderBy("value", "partition")
    .show()
)

+-----+---------+------+
|value|partition| count|
+-----+---------+------+
|    0|        0|332774|
|    0|        1|333601|
|    0|        2|333615|
|    1|        0|     6|
|    1|        1|     9|
|    2|        0|     2|
|    2|        1|     2|
|    2|        2|     6|
|    3|        0|     3|
|    3|        1|     2|
+-----+---------+------+



# Salting In Aggregations

In [ ]:
df_skew.groupBy("value").count().show()

+-----+------+
|value| count|
+-----+------+
|    0|999990|
|    2|    10|
|    3|     5|
|    1|    15|
+-----+------+



In [ ]:
(
    df_skew
    .withColumn("salt", (F.rand() * SALT_NUMBER).cast("int"))
    .groupBy("value", "salt")
    .agg(F.count("value").alias("count"))
    .groupBy("value")
    .agg(F.sum("count").alias("count"))
    .show()
)

In [ ]:
spark.stop()